In [1]:
from Crypto.Cipher import AES
import scrypt, os, binascii
import csv
import itertools

In [2]:
def generate_key(passwd, N=16384, r=8, p=1, sklen=32, saltlen=16):
    """
    * passwd:              str or str encoded as bype object
    * saltlen:             8 bytes minimum, 16 bytes recommended
    * salt:                saltlen-byte (non-secret) random data for scrypt key derivation function
    * secretKey:           bytes object of length sklen
        * N:               iterations count (a power of 2 greater than 1) (e.g. 16384, 2048)
        * r:               block size 
        * p:               parallelism factor (where r*p < 2^30)
        * sklen:           no. of bytes to generate as output (<= (2**32 - 1)*32)
    * Memory required = 128 * N * r * p bytes
    """
    if type(passwd) == str:
        passwd = passwd.encode()
    salt = os.urandom(saltlen)
    secretKey = scrypt.hash(passwd, salt, N, r, p, sklen)
    return secretKey, salt
    
def encrypt_AES_GCM(msg, secretKey, salt):
    '''
    * msg:                 str or str encoded as bype object
    * aesCipher:           AES-256 cipher in Galois/Counter Mode
    * aesCipher.nonce:     nonce bype object
    * ciphertext, authTag: ciphertext and MAC tag
    '''
    if type(msg) == str:
        msg = msg.encode()
    aesCipher = AES.new(secretKey, AES.MODE_GCM)
    ciphertext, authTag = aesCipher.encrypt_and_digest(msg)
    return (salt, ciphertext, aesCipher.nonce, authTag)

def decrypt_AES_GCM(encryptedMsg, secretKey):
    """
    * encryptedMsg:        quadruple (salt, ciphertext, nonce, authTag)
    * plaintext:           plaintext as byte object
    """
    (salt, ciphertext, nonce, authTag) = encryptedMsg
    aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce)
    plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
    return plaintext

In [259]:
def encrypt_CSV(file_name, KEK, N=16384, r=8, p=1, sklen=32, saltlen=16, return_DEKs = False):
    # @TODO: more efficient: store each DEK info once, not with each entry
    file = open(file_name, 'r')
    reader_, reader = itertools.tee(csv.reader(file, delimiter=','))
    columns = len(next(reader_))
    del reader_
    
    DEKs = [] # one per column
    for _ in range(columns):
        DEKs += [generate_key(KEK, N, r, p, sklen)]
    """
    with open(file_name[:-4] + '_keyinfo.csv', 'w') as key_file:
        fieldnames = ['salt', 'ciphertext', 'aesCipher.nonce', 'authTag']
        writer = csv.DictWriter(key_file, fieldnames = fieldnames) etc. etc.
    """

    header = next(reader) 
    encrypted_data = []
    for line in reader:
        if line:
            encrypted_row = {}
            for j in range(len(line)):
                encrypted_row[header[j]] = encrypt_AES_GCM(line[j], DEKs[j][0], DEKs[j][1])
            encrypted_data.append(encrypted_row)
            
    with open(file_name[:-4] + '_enc.csv', 'w') as enc_file:
        writer = csv.DictWriter(enc_file, fieldnames = header)
        writer.writeheader()
        for row in encrypted_data:
            writer.writerow(row)
    file.close()
    if return_DEKs:
        return DEKs # for debugging purposes
    
def decrypt_CSV(enc_file_name, KEK, N=16384, r=8, p=1, sklen=32, saltlen=16):
    file = open(enc_file_name, 'r')
    reader = csv.reader(file)
    
    header = next(reader) 
    decrypted_data = []
    for line in reader:
        encrypted_row = []
        for entry in line:
            entry = entry[3:-2].split("', b'")
            encryptedMsg = [w.encode() for w in entry]
            secretKey = scrypt.hash(KEK, encryptedMsg[0], N, r, p, sklen)
            
            print(secretKey)
            print(encryptedMsg[0]) 
            print(encryptedMsg[0] == DEKs[0][1]) # False becaust of the extra \'s?
            return
            # decryptedMsg = decrypt_AES_GCM(encryptedMsg, secretKey)

    # @TODO: print to file
    file.close()

* * *

# Sandbox

Encrypt and decrypt CSV: (Not working; seems to be retrieving the incorrect DEK).

In [260]:
KEK, KEK_salt = generate_key("myP@ssw0rd!")
DEKs = encrypt_CSV('biostats.csv', KEK, return_DEKs = True)

In [261]:
print(DEKs[0][0])
print(DEKs[0][1])

b'\xdap\xd1\xf6\xa7\x83x\xcd\x83\xe4\x8ev\xf3\x94\x88.Y/\x92\xde\x18\x01\xd9`1z\xb3A\x89k\nO'
b'\x18\x8a\xa1\xd4\xa7\x0f\xf4\x9d\xce\xfc\x1a44\xd1u\xb5'


In [262]:
scrypt.hash(KEK, DEKs[0][1])

b'\xdap\xd1\xf6\xa7\x83x\xcd\x83\xe4\x8ev\xf3\x94\x88.Y/\x92\xde\x18\x01\xd9`1z\xb3A\x89k\nOg\x88\xf8&\xb4e\x00\xe1\x81\xcc\x1eUy\x06\x9a\xdfB_\xccMeV\xca\xc5\x02B\xd4\x88\xdc\xdc\x8a\xcf'

In [263]:
decrypt_CSV('biostats_enc.csv', KEK)

b'\xee\x9eA\x9a\xd7\x8e>\x0b\xaf\xf9\xaa)b\x8eev\x06\xcb\xd3\xb60\x99\x81\xb8\xc1(\xbe\x94\xa9\x02\x9c)'
b'\\x18\\x8a\\xa1\\xd4\\xa7\\x0f\\xf4\\x9d\\xce\\xfc\\x1a44\\xd1u\\xb5'
False


* * *

Encrypt and decrypt simple message:

In [4]:
msg = "The Hitchhiker's Guide to the Galaxy has a few things to say on the subject of towels. A towel, it says, is about the most massively useful thing an interstellar hitch hiker can have. Partly it has great practical value—you can wrap it around you for warmth as you bound across the cold moons of Jaglan Beta; you can lie on it on the brilliant marble‐sanded beaches of Santraginus V, inhaling the heady sea vapours; you can sleep under it beneath the stars which shine so redly on the desert world of Kakrafoon; use it to sail a mini raft down the slow heavy river Moth; wet it for use in hand‐to‐hand‐combat; wrap it round your head to ward off noxious fumes or to avoid the gaze of the Ravenous Bugblatter Beast of Traal (a mindbogglingly stupid animal, it assumes that if you can't see it, it can't see you—daft as a brush, but very ravenous); you can wave your towel in emergencies as a distress signal, and of course dry yourself off with it if it still seems to be clean enough."
#msg = ['All`s Well That Ends Well', 'As You Like It', 'Comedy of Errors', 'Love`s Labour`s Lost', 'Measure for Measure', 'Merchant of Venice', 'Merry Wives of Windsor', 'Midsummer Night`s Dream', 'Much Ado about Nothing', 'Taming of the Shrew', 'Tempest', 'Twelfth Night', 'Two Gentlemen of Verona', 'Winter`s Tale']
password = "myP@ssw0rd!"

DEK, salt = generate_key(password)
encryptedMsg = encrypt_AES_GCM(msg, DEK, salt)
print("encryptedMsg", {
    'kdfSalt': binascii.hexlify(encryptedMsg[0]),
    'ciphertext': binascii.hexlify(encryptedMsg[1]),
    'aesIV': binascii.hexlify(encryptedMsg[2]),
    'authTag': binascii.hexlify(encryptedMsg[3])
})

encryptedMsg {'kdfSalt': b'890518ef840f2629f88b1f59b16558d3', 'ciphertext': b'5ed016c8f6e3ef0cd7b48708c17b6b7486c01d2efdba00f6e41b1a871e5799e570315d7771ddbe4636464963336c8c05ab2cb1f7719004d7cb7ce94d9d3cbc2258fad96976afabbd490a9e98e6793a3f3d10acc5ac820b7f7081cbb61861078981164b7c4161ef0e87d5b437cbe89323b826ef8388699ca0989f0b1362a8f90f45f4c5a389f5ac5d3a92abba9e6f431377eeeff24c9278ae8e1ad817dd838bd69ffb48699894b663addd1fbb030e9383f9e3a126760be9a0ff0addb4e048ca38e5d570a2ceb0811c9fee16084e7dc72cac698d77a951113e85c65e16a17a0844ebc5879f13506516390d47a933c2a7aba093cb3cf429863fbb06389eeb105b7f7cbf050478f66755c20571ff43f7058d59469943dcee422aeb45682222b46cc064a43590f0b3d1881ce82639db0981e49c480c6db9db5a4e27a493b39d4a63d7a935aab67c3848b1357f077182763eee08c956e2e61e3cd0b8a59c9396f7ff897f813ed0810a47af4157fca853c521e6fc656c3452acf5ed1fddf04b1c98e6f7dae24b9ddf8e81206c864521eb0461de5bd3d8dc9943f5d7a455d438dad2206c82b95fb9742e8a3ca85d3636762f4ee9bd3c62beb126414eab781c35000033a58c73009c2d5d978ef728dc17f1

In [5]:
decryptedMsg = decrypt_AES_GCM(encryptedMsg, DEK)
print("Decrypted message:", decryptedMsg.decode())

Decrypted message: The Hitchhiker's Guide to the Galaxy has a few things to say on the subject of towels. A towel, it says, is about the most massively useful thing an interstellar hitch hiker can have. Partly it has great practical value—you can wrap it around you for warmth as you bound across the cold moons of Jaglan Beta; you can lie on it on the brilliant marble‐sanded beaches of Santraginus V, inhaling the heady sea vapours; you can sleep under it beneath the stars which shine so redly on the desert world of Kakrafoon; use it to sail a mini raft down the slow heavy river Moth; wet it for use in hand‐to‐hand‐combat; wrap it round your head to ward off noxious fumes or to avoid the gaze of the Ravenous Bugblatter Beast of Traal (a mindbogglingly stupid animal, it assumes that if you can't see it, it can't see you—daft as a brush, but very ravenous); you can wave your towel in emergencies as a distress signal, and of course dry yourself off with it if it still seems to be clean enou

***

Encrypt and decrypt text file (incomplete):

In [200]:
def encrypt_file(file_name, secretKey, salt):
    with open(file_name, 'rb') as file:
        plaintext = file.read()
    (salt, ciphertext, nonce, authTag) = encrypt_AES_GCM(plaintext, secretKey, salt)
    # with open(file_name + ".enc", 'wb') as f:
    #     f.write(salt + b'\n')
    #     f.write(ciphertext + b'\n')
    #     f.write(nonce + b'\n')
    #     f.write(authTag + b'\n')  
    # this can be done more efficiently
    with open(file_name + ".salt.txt", 'wb') as f:
        f.write(salt)
    with open(file_name + ".enc", 'wb') as f:
        f.write(ciphertext)
    with open(file_name + ".nonce.txt", 'wb') as f:
        f.write(nonce)
    with open(file_name + ".authTag.txt", 'wb') as f:
        f.write(authTag)  
            
def decrypt_file(file_name, secretKey):
    with open(file_name[:-4] + ".salt.txt", 'rb') as f:
        salt = f.read()
    with open(file_name) as f:
        ciphertext = f.read()
    with open(file_name[:-4] + ".nonce.txt", 'rb') as f:
        nonce = f.read()
    with open(file_name[:-4] + ".authTag.txt", 'rb') as f:
        authTag = f.read()
    (salt, ciphertext, nonce, authTag) = encryptedMsg
    plaintext = decrypt_AES_GCM(encryptedMsg, secretKey)
    with open(file_name[:-4], 'wb') as f:
        f.write(plaintext)

In [201]:
password = "myP@ssw0rd!"
DEK, salt = generate_key(password)
encrypt_file('/Users/dzackon/Desktop/testfile.txt', DEK, salt)

In [ ]:
decrypt_AES_GCM('/Users/dzackon/Desktop/testfile.txt.enc', DEK)